In [67]:
import random
import numpy as np
import json
from tqdm import tqdm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import linear_sum_assignment
from collections import Counter


In [68]:
def generate_synthetic_data(params):
    '''
    Synthetic Data Generation
    '''
    # Dense K: matrix of binary images of sizeNxDxM
    # Sparse K: set of (delay d, neuron a, and pg b)

    M,N,D,T,nrn_fr,pg_fr,background_noise_fr = params['M'], params['N'], params['D'], params['T'], params['nrn_fr'],params['pg_fr'],params['background_noise_fr'],    
    '''
    Synthetic Data Generation
    '''
    # Dense K: matrix of binary images of sizeNxDxM
    # Sparse K: set of (delay d, neuron a, and pg b)

    K_dense = np.random.rand(N,D,M)*1000
    nrn_frs = np.zeros((M))
    for m in range(M):
        nrn_frs[m] = np.random.poisson(nrn_fr)
        K_dense[:,:,m] = (K_dense[:,:,m] < nrn_frs[m]).astype('int')
    K_sparse = np.where(K_dense)
    K_sparse = (K_sparse[0],K_sparse[1],K_sparse[2]+1)


    # dense B: the binary image of the occurrences of the spiking motif as a ( M x T) matrix
    # spare B: set of all times t and pg's b
    B_dense = np.random.rand(M,T)*1000
    pg_frs = np.zeros((M))
    for m in range(M):
        pg_frs[m] = np.random.poisson(pg_fr)
        B_dense[m,:] = (B_dense[m,:] < pg_frs[m]).astype('int')
    B_sparse = np.where(B_dense)
    B_sparse = (B_sparse[0]+1,B_sparse[1])# This way the first motif starts at index 1 instead of index 0

    # now to make the full raster plot keeping the labels in-tact
    # dense A: the layered binary images of all neuron spikes by PG ( N x T x M
    A_dense = np.zeros((N,T+D,M+1))
    A_dense[...,0] = np.random.rand(N,T+D)*1000
    A_dense[...,0] = (A_dense[...,0] < background_noise_fr).astype('int')
    for i in range(len(B_sparse[0])):
        t = B_sparse[1][i]
        b = B_sparse[0][i]
        A_dense[:, t:t+D, b] += K_dense[...,b-1]

    A_sparse = np.where(A_dense)
    A_dense = np.sum(A_dense,axis=2)
    A_dense[A_dense>1] = 1
    return A_dense, A_sparse, B_dense, B_sparse, K_dense, K_sparse

In [413]:
def scan_raster(T_labels, N_labels, window_dim = None):
    '''
    T_labels an array of spiketimes
    N_labels corresponding array of neuron labels
    window_dim is the size of the window to cluster the spikes
    '''
    print(f'Cleaning spikes...',end='\r')
    if window_dim == None:
        window_dim = 100
        
    T_labels = np.round(T_labels).astype(int)
    T_labels, N_labels = np.unique(np.array([T_labels,N_labels]),axis=1) # This removes any spikes that occur at the same neuron at the same time
    N=max(N_labels)+1

    print(f'Windowing... {len(T_labels)}')
    windows = np.zeros((len(T_labels)),dtype='object')
    for i,window_time in enumerate(T_labels):
        condition = (T_labels > window_time-window_dim) & (T_labels < window_time + window_dim)
        window = np.array([T_labels[condition]-window_time, N_labels[condition]]).T
        window =  {tuple(row) for row in  window}
        windows[i] = window

        
    # Set the cutoff value for clustering
    cutoff = 0
    lr = 0.01

    max_iter=50
    lr = 0.01
    iter_ = 0

    opt_cutoff = cutoff
    max_seq_rep = 0
    sim_mats = _get_sim_mats(windows, T_labels, N_labels)

    while iter_ <= max_iter: # this is just a for loop...
        clusters = _cluster_windows(cutoff, N_labels, sim_mats)
        cluster_sq, _sq_counts, sublist_keys_filt = _check_seq(clusters, T_labels, N_labels)

        if len(sublist_keys_filt) != 0:
            max_ = np.max([len(k) for k in sublist_keys_filt])
            if max_seq_rep < max_:
                max_seq_rep = max_
                opt_cutoff=cutoff

        cutoff += lr
        iter_ +=1


        print(f'iter - {iter_/max_iter} | cutoff - {cutoff} | opt_cutoff - {opt_cutoff} | most_detections - {max_seq_rep}',end='\r')

    clusters = _cluster_windows(opt_cutoff, N_labels, sim_mats)
    cluster_sq, sq_counts, sublist_keys_filt = _check_seq(clusters, T_labels, N_labels)
        

    ''' to get the timings'''

    # Sort y according to x
    sorted_indices = np.argsort(T_labels)
    sorted_x = T_labels[sorted_indices]

    all_times = []
    all_labels = []
    for key in sublist_keys_filt:
        pattern_repetition_labels = np.zeros((len(cluster_sq[str(key)]),len(clusters)))
        for i,k in enumerate(cluster_sq[str(key)]):
            pattern_repetition_labels[i][clusters==k] = 1
            pattern_repetition_labels[i] *= np.cumsum(pattern_repetition_labels[i])
        pattern_repetition_labels = np.sum(pattern_repetition_labels,axis=0,dtype='int')
        all_labels.append(pattern_repetition_labels)

        sorted_y = pattern_repetition_labels[sorted_indices]
        pattern_times = np.array([sorted_x[sorted_y==i][0] for i in range(1,max(pattern_repetition_labels)+1)])
        all_times.append(pattern_times)

    pattern_template = []
    patterns = []
    for i in range(len(all_times)):
        pattern = []
        pattern_template.append([])
        for time in all_times[i]:
            condition = (T_labels > time-window_dim*2) & (T_labels < time + window_dim*2)
            pattern = [tuple(k) for k in np.array([T_labels[condition]-time, N_labels[condition]]).T] # creating a list of tuples
            pattern_template[-1] += pattern # adds all points of each pattern to template_pattern
            patterns.append(pattern)

    for i,pattern in enumerate(pattern_template):
        counts = [pattern.count(k) for k in pattern]
        pattern_template[i] = np.array(pattern)[np.where(counts == np.max(counts))[0]]
        pattern_template[i][:,0] -= min(pattern_template[i][:,0])
        pattern_template[i] = np.unique(pattern_template[i],axis=0)
    
    if len(pattern_template) == 0:
        return pattern_template, sublist_keys_filt, None
    
    win_size = (N,1+max([max(k[:,0]) for k in pattern_template]))
    pattern_img = np.zeros((len(pattern_template),*win_size))
    for p,pattern in enumerate(pattern_template):
        for (i,j) in pattern:
            pattern_img[p,j,i] = 1

    return pattern_template, sublist_keys_filt, pattern_img

def _get_sim_mats(windows, T_labels, N_labels):
    intersect = lambda a,b : a[((a[:,None] == B).all(-1).any(1))]
    sim_mats = np.zeros(N,dtype='object')
    for n in np.unique(N_labels):
        idc = np.where(N_labels==n)[0]
        windows_n = windows[idc]
        if len(windows_n) > 1:
            x = np.zeros((len(windows_n),len(windows_n)))
            for i in range(windows_n.shape[0]):
                for j in range(windows_n.shape[0]):
                    common_rows = windows_n[i].intersection(windows_n[j])
                    num_identical_rows = len(common_rows)
                    x[i,j] = len(common_rows)/min(len(windows_n[i]),len(windows_n[j]))
            np.fill_diagonal(x,0)# make sure the diagonals are zero, this is important the more spikes there are...
            sim_mats[n] = x-1 
    return sim_mats

def _cluster_windows(cutoff, N_labels, sim_mats):
    clusters = np.zeros_like(N_labels)
    for n in np.unique(N_labels):
        idc = np.where(N_labels==n)[0]
        if (type(sim_mats[n]) == np.ndarray) and (not np.all(sim_mats[n] == 0)):
            l = max(clusters)+1
            clusters[idc]= l+fcluster(linkage(sim_mats[n], method='complete'), cutoff, criterion='distance')
    return clusters

def _check_seq(clusters, T_labels, N_labels):

    time_differences = []
    cluster_sq = {}
    for cluster in np.unique(clusters):
        temp = list(np.diff(np.unique(T_labels[clusters == cluster])))
        str_temp = str(temp)
        time_differences.append(temp)
        if str_temp in cluster_sq.keys():
            cluster_sq[str_temp] = cluster_sq[str_temp] + [cluster]
        else:
            cluster_sq[str_temp] = [cluster]

    # Convert the list of lists to a set of tuples to remove duplicates
    unique_sublists_set = set(tuple(sublist) for sublist in time_differences if sublist)

    # Convert the set of tuples back to a list of lists
    unique_sublists = [list(sublist) for sublist in unique_sublists_set]

    # Count the occurrences of each unique sublist in the original list
    sublist_counts = Counter(tuple(sublist) for sublist in time_differences if sublist)

    # Print the unique sublists and their respective counts
    sq_counts = np.zeros(len(sublist_counts)) 
    for i,sublist in enumerate(unique_sublists):
        count = sublist_counts[tuple(sublist)]
        sq_counts[i] = count
    #     print(f"{sublist}: {count} occurrences")
    sublist_keys_np = np.array([list(key) for key in sublist_counts.keys()],dtype='object')
    sublist_keys_filt = sublist_keys_np[np.array(list(sublist_counts.values())) >1] # only bother clustering repetitions that appear for more than one neuron
    
    return cluster_sq, sq_counts, sublist_keys_filt


In [408]:
from scipy.signal import correlate
def get_acc(ground_truths,detected_patterns):
    # Calculate cross-correlation matrix
    cross_corr_matrix = np.zeros((ground_truths.shape[2], detected_patterns.shape[2]))
    SM_acc = np.zeros((ground_truths.shape[2]))
    
    if len(detected_patterns == 0):
        return SM_acc, cross_corr_matrix
    
    for ground_truths_idx in range(ground_truths.shape[2]):
        for detected_patterns_idx in range(detected_patterns.shape[2]):
            cross_corr = np.zeros((ground_truths.shape[1]+detected_patterns.shape[1]-1))
            for n in range(ground_truths.shape[0]):
                cross_corr += correlate(ground_truths[n, :, ground_truths_idx], detected_patterns[n, :, detected_patterns_idx], mode='full')
            max_corr = np.max(cross_corr) / max(np.sum(ground_truths[...,ground_truths_idx]),np.sum(detected_patterns[...,detected_patterns_idx]))
            cross_corr_matrix[ground_truths_idx, detected_patterns_idx] = max_corr
#     print(cross_corr_matrix)
#     print( np.sum(ground_truths[...,ground_truths_idx]))
    SM_acc = np.max(cross_corr_matrix,axis=1)
    return SM_acc, cross_corr_matrix

In [409]:
def main():
        # Define the number of random samples you want to take
    num_samples = 5  # Adjust this based on your computational resources
    
    trials = 1
    
    # List to hold the results
    results = []
    
    param_combinations = np.array(np.meshgrid(*scan_dict.values())).T.reshape(-1, len(scan_dict))
    num_iterations = len(param_combinations)
    
    # Generate random indices for sampling
    random_indices = random.sample(range(num_iterations), num_samples)
    
    # Iterate through parameter combinations
    for idx in tqdm(random_indices):
        for trial in range(0,trials):
            seed=trial
            np.random.seed(seed)
            params = {key: int(val) for key, val in zip(scan_dict.keys(), param_combinations[idx])}

            # Run your program here to generate performance results
            print("Params:", params)
            print("Generating raster plot...")
            _, A_sparse, _, B_sparse, K_dense, K_sparse = generate_synthetic_data(params)
            print("Clustering...")
            pattern_template, sublist_keys_filt, pattern_img = scan_raster(A_sparse[1],A_sparse[0],window_dim=params['D'])
            if type(pattern_img) != np.ndarray:
                performance_result = (0,0)
            else:
                pattern_img = np.transpose(pattern_img,axes=[1,2,0])
                SM_acc, _ = get_acc(K_dense, pattern_img)
                performance_result = (np.sum(SM_acc>0.8)/len(SM_acc), np.mean(SM_acc))

            # Create a dictionary to store the result
            result = {
                'idc': idx,
                'trial':trial,
                'data':[A_sparse,K_sparse,B_sparse],
                **params,  # Unpack the parameters as separate columns
                'performance':performance_result
            }
            
            print(performance_result)

            # Append the result to the list
            results.append(result)
        # Write the entire list of results to a JSON file
    with open('scan_stats.json', 'w') as results_file:
        json.dump(results, results_file, indent=4)
        
    return results


In [410]:
'''
Model default parameters
'''

M = 4 # Number of Spiking motifs
N = 20 # Number of input neurons
D = 71 # temporal depth of receptive field
T = 1000
nrn_fr = 15 # hz
pg_fr = 6 # hz
background_noise_fr = 10 # h
seed=41
np.random.seed(seed)

In [411]:
default_params = {
    'M':M,
    'N':N,
    'D':D,
    'T':T,
    'nrn_fr':nrn_fr,
    'pg_fr':pg_fr,
    'background_noise_fr':background_noise_fr,
    'seed':seed
}
scan_dict = {
    'M':[1,4,16,32,64],
    'N':[5,30,60,100],
    'D':[10,30,70,150],
    'T':[500,1000,1500],
    'nrn_fr':[5,10,15,20],
    'pg_fr':[3,4,5,8,10,20],
    'background_noise_fr':[0,1,2,5,10,20]
}

In [412]:
results = main()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Params: {'M': 16, 'N': 60, 'D': 30, 'T': 1000, 'nrn_fr': 15, 'pg_fr': 10, 'background_noise_fr': 5}
Generating raster plot...
Clustering...
Windowing... [   2    8    9 ... 1028 1029 1029]


  0%|                                                                                            | 0/5 [00:06<?, ?it/s]


IndexError: index 20 is out of bounds for axis 0 with size 20

In [354]:
for i in range(windows_n.shape[0]):
        for j in range(windows_n.shape[0]):
            common_rows = intersect(windows_n[i],windows_n[j])
            num_identical_rows = len(common_rows)
            x[i,j] = len(common_rows)/min(len(windows_n[i]),len(windows_n[j]))
        print(i)

0
1
2
3
4
5
6
7


KeyboardInterrupt: 

In [ ]:
window1 =  {tuple(row) for row in  window}
window1 =  {tuple(row) for row in  window}

In [380]:
len(T_labels)

100

In [402]:
T_labels.astype(np.int16)

array([  24,   45,   55, ..., 1095, 1114, 1125], dtype=int16)

In [406]:
start = time.time()
# T_labels,N_labels = A_sparse[1],A_sparse[0]
# windows = np.zeros((len(T_labels)),dtype='object')
# for i,window_time in enumerate(T_labels):
#     condition = (T_labels > window_time-window_dim) & (T_labels < window_time + window_dim)
#     window = np.array([T_labels[condition]-window_time, N_labels[condition]]).T
#     window =  {tuple(row) for row in  window}
#     windows[i] = window
#     print(i/len(T_labels),end='\r')
windows.tolist()
sim_mats = np.zeros(N,dtype='object')
for n in [0]:#np.unique(N_labels):
    idc = np.where(N_labels==n)[0]
    
    windows_n = windows[idc]
    if len(windows_n) > 1:
        x = np.zeros((len(windows_n),len(windows_n)))
        for i in range(windows_n.shape[0]):
            for j in range(windows_n.shape[0]):
                common_rows = windows_n[i].intersection(windows_n[j])
                num_identical_rows = len(common_rows)
                x[i,j] = len(common_rows)/min(len(windows_n[i]),len(windows_n[j]))
        np.fill_diagonal(x,0)# make sure the diagonals are zero, this is important the more spikes there are...
        sim_mats[n] = x-1 
end = time.time()
print(end - start)

22.835694551467896


In [399]:
start = time.time()
T_labels,N_labels = A_sparse[1],A_sparse[0]
T_labels, N_labels = T_labels, N_labels
windows = np.zeros((len(T_labels)),dtype='object')
for i,window_time in enumerate(T_labels):
    condition = (T_labels > window_time-window_dim) & (T_labels < window_time + window_dim)
    window = np.array([T_labels[condition]-window_time, N_labels[condition]]).T
    window =  {tuple(row) for row in  window}
    windows[i] = window
    print(i/len(T_labels),end='\r')
sim_mats = np.zeros(N,dtype='object')
for n in [0]:#np.unique(N_labels):
    idc = np.where(N_labels==n)[0]
    
    windows_n = windows[idc]
    if len(windows_n) > 1:
        x = np.zeros((len(windows_n),len(windows_n)))
        for i in range(windows_n.shape[0]):
            for j in range(windows_n.shape[0]):
                common_rows = windows_n[i].intersection(windows_n[j])
                num_identical_rows = len(common_rows)
                x[i,j] = len(common_rows)/min(len(windows_n[i]),len(windows_n[j]))
        np.fill_diagonal(x,0)# make sure the diagonals are zero, this is important the more spikes there are...
        sim_mats[n] = x-1 
end = time.time()
print(end - start)

384.950688838958745655


In [382]:

start = time.time()
T_labels,N_labels = A_sparse[1],A_sparse[0]
T_labels, N_labels = T_labels[:1000], N_labels[:1000]
windows = np.zeros((len(T_labels)),dtype='object')
for i,window_time in enumerate(T_labels):
    condition = (T_labels > window_time-window_dim) & (T_labels < window_time + window_dim)
    window = np.array([T_labels[condition]-window_time, N_labels[condition]]).T
    windows[i] = window
intersect = lambda a,b : a[((a[:,None] == B).all(-1).any(1))]
sim_mats = np.zeros(N,dtype='object')
for n in [0]:#np.unique(N_labels):
    idc = np.where(N_labels==n)[0]
    windows_n = windows[idc]
    if len(windows_n) > 1:
        x = np.zeros((len(windows_n),len(windows_n)))
        for i in range(windows_n.shape[0]):
            for j in range(windows_n.shape[0]):
                common_rows = intersect(windows_n[i],windows_n[j])
                num_identical_rows = len(common_rows)
                x[i,j] = len(common_rows)/min(len(windows_n[i]),len(windows_n[j]))
        np.fill_diagonal(x,0)# make sure the diagonals are zero, this is important the more spikes there are...
        sim_mats[n] = x-1 
end = time.time()
print(end - start)

53.30270743370056


In [385]:
A = windows_n[0]
B = windows_n[1]
intersect = lambda a,b : a[((a[:,None] == B).all(-1).any(1))]
intersect(A,B)

array([[  0,   0],
       [ 31,   0],
       [ 77,   0],
       [ 92,   0],
       [ 98,   0],
       [-12,   2],
       [ 77,   3],
       [ 95,   3],
       [ 95,   3],
       [ 98,   3],
       [ 99,   3],
       [ 88,   4]], dtype=int64)

In [388]:
A.T

array([[  0,  21,  31,  43,  52,  60,  66,  67,  77,  84,  90,  92,  93,
         94,  98,  22,  23,  35,  42,  47,  62,  74,  93,  97,  99, -12,
          9,  22,  35,  47,  48,  70,  81,  86,  87,  96,  96,  26,  33,
         45,  49,  57,  58,  61,  73,  77,  84,  85,  95,  95,  97,  98,
         99,  -6,   3,  43,  54,  62,  72,  84,  88,  90],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   2,
          2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   3,   3,
          3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,
          3,   4,   4,   4,   4,   4,   4,   4,   4,   4]], dtype=int64)

In [393]:
np.intersect1d(A.T[0],B.T[0],return_indices=True)

(array([-12,   0,  21,  22,  26,  31,  33,  45,  49,  52,  60,  66,  67,
         72,  73,  74,  77,  81,  86,  87,  88,  90,  92,  95,  97,  98,
         99], dtype=int64),
 array([25,  0,  1, 15, 37,  2, 38, 39, 40,  4,  5,  6,  7, 58, 44, 21,  8,
        32, 33, 34, 60, 10, 11, 48, 23, 14, 24], dtype=int64),
 array([36,  1, 21,  3, 22,  4, 75,  6, 41, 54, 42, 44, 79, 12, 13, 58, 14,
        65, 30, 31, 32, 33, 15, 69, 16, 17, 71], dtype=int64))

In [396]:
np.intersect1d(A.T[1],B.T[1],return_indices=True)

(array([0, 1, 2, 3, 4], dtype=int64),
 array([ 0, 15, 25, 37, 53], dtype=int64),
 array([ 0, 18, 35, 47, 72], dtype=int64))

In [397]:
idc1 = np.intersect1d(A.T[0],B.T[0],return_indices=True)[1]
idc2 = np.intersect1d(A.T[1],B.T[1],return_indices=True)[1]
idc3 = np.intersect1d(idc1, idc2)
idc3

array([ 0, 15, 25, 37], dtype=int64)

In [365]:
intersect = lambda a,b : a[((a[:,None] == B).all(-1).any(1))]
start = time.time()
sim_mats = np.zeros(N,dtype='object')
for n in [0]:#np.unique(N_labels):
    idc = np.where(N_labels==n)[0]
    windows_n = windowsset[0]
    if len(windows_n) > 1:
        x = np.zeros((len(windows_n),len(windows_n)))
        for i in range(windows_n.shape[0]):
            for j in range(windows_n.shape[0]):
                common_rows = windows_n[i].intersection(windows_n[j])
                num_identical_rows = len(common_rows)
                x[i,j] = len(common_rows)/min(len(windows_n[i]),len(windows_n[j]))
        np.fill_diagonal(x,0)# make sure the diagonals are zero, this is important the more spikes there are...
        sim_mats[n] = x-1 
end = time.time()
print(end - start)


TypeError: unhashable type: 'numpy.ndarray'

In [227]:
if window_dim == None:
    window_dim = 100

T_labels = np.round(T_labels).astype(int)
T_labels, N_labels = np.unique(np.array([T_labels,N_labels]),axis=1) # This removes any spikes that occur at the same neuron at the same time
N=max(N_labels)+1

print(f'Windowing... {len(T_labels)}')
windows = np.zeros((len(T_labels)),dtype='object')
for i,window_time in enumerate(T_labels):
    condition = (T_labels > window_time-window_dim) & (T_labels < window_time + window_dim)
    window = np.array([T_labels[condition]-window_time, N_labels[condition]]).T
    window =  {tuple(row) for row in  window}
    windows[i] = window
    print(f'{np.round(i/len(T_labels),1)}',end='\r')


# Set the cutoff value for clustering
cutoff = 0
lr = 0.01

max_iter=50
lr = 0.01
iter_ = 0

opt_cutoff = cutoff
max_seq_rep = 0
sim_mats = _get_sim_mats(windows, T_labels, N_labels)

while iter_ <= max_iter: # this is just a for loop...
    clusters = _cluster_windows(cutoff, N_labels, sim_mats)
    cluster_sq, _sq_counts, sublist_keys_filt = _check_seq(clusters, T_labels, N_labels)

    if len(sublist_keys_filt) != 0:
        max_ = np.max([len(k) for k in sublist_keys_filt])
        if max_seq_rep < max_:
            max_seq_rep = max_
            opt_cutoff=cutoff

    cutoff += lr
    iter_ +=1


    print(f'iter - {iter_/max_iter} | cutoff - {cutoff} | opt_cutoff - {opt_cutoff} | most_detections - {max_seq_rep}',end='\r')

clusters = _cluster_windows(opt_cutoff, N_labels, sim_mats)
cluster_sq, sq_counts, sublist_keys_filt = _check_seq(clusters, T_labels, N_labels)


''' to get the timings'''

# Sort y according to x
sorted_indices = np.argsort(T_labels)
sorted_x = T_labels[sorted_indices]

all_times = []
all_labels = []
for key in sublist_keys_filt:
    pattern_repetition_labels = np.zeros((len(cluster_sq[str(key)]),len(clusters)))
    for i,k in enumerate(cluster_sq[str(key)]):
        pattern_repetition_labels[i][clusters==k] = 1
        pattern_repetition_labels[i] *= np.cumsum(pattern_repetition_labels[i])
    pattern_repetition_labels = np.sum(pattern_repetition_labels,axis=0,dtype='int')
    all_labels.append(pattern_repetition_labels)

    sorted_y = pattern_repetition_labels[sorted_indices]
    pattern_times = np.array([sorted_x[sorted_y==i][0] for i in range(1,max(pattern_repetition_labels)+1)])
    all_times.append(pattern_times)

pattern_template = []
patterns = []
for i in range(len(all_times)):
    pattern = []
    pattern_template.append([])
    for time in all_times[i]:
        condition = (T_labels > time-window_dim*2) & (T_labels < time + window_dim*2)
        pattern = [tuple(k) for k in np.array([T_labels[condition]-time, N_labels[condition]]).T] # creating a list of tuples
        pattern_template[-1] += pattern # adds all points of each pattern to template_pattern
        patterns.append(pattern)

for i,pattern in enumerate(pattern_template):
    counts = [pattern.count(k) for k in pattern]
    pattern_template[i] = np.array(pattern)[np.where(counts == np.max(counts))[0]]
    pattern_template[i][:,0] -= min(pattern_template[i][:,0])
    pattern_template[i] = np.unique(pattern_template[i],axis=0)

if len(pattern_template) == 0:
    break

win_size = (N,1+max([max(k[:,0]) for k in pattern_template]))
pattern_img = np.zeros((len(pattern_template),*win_size))
for p,pattern in enumerate(pattern_template):
    for (i,j) in pattern:
        pattern_img[p,j,i] = 1

Windowing... 22437


AttributeError: 'numpy.ndarray' object has no attribute 'intersection'

In [138]:
sim_mats[n]

IndexError: index 99 is out of bounds for axis 0 with size 60

In [141]:
clusters = np.zeros_like(N_labels)
for n in np.unique(N_labels):
    idc = np.where(N_labels==n)[0]
    if (len(sim_mats[n]) > 0) and (not np.all(sim_mats[n] == 0)):
        l = max(clusters)+1
        clusters[idc]= l+fcluster(linkage(sim_mats[n], method='complete'), cutoff, criterion='distance')


TypeError: object of type 'int' has no len()

In [144]:
sim_mats[20]

array([[-1.        , -0.96428571, -0.98214286, -0.98214286, -0.96428571,
        -0.94642857, -0.96428571, -0.98214286, -0.98214286, -0.98214286,
        -0.625     , -0.98214286, -0.98214286],
       [-0.96428571, -1.        , -0.96062992, -0.992     , -0.82075472,
        -0.98181818, -0.85      , -0.79545455, -0.98648649, -0.98245614,
        -0.96875   , -0.84375   , -0.984375  ],
       [-0.98214286, -0.96062992, -1.        , -0.832     , -0.96226415,
        -0.8       , -0.95833333, -0.97727273, -0.97297297, -0.59649123,
        -0.82677165, -0.96850394, -0.96062992],
       [-0.98214286, -0.992     , -0.832     , -1.        , -0.99056604,
        -0.79090909, -0.975     , -0.98863636, -0.97297297, -0.63157895,
        -0.824     , -0.992     , -0.96      ],
       [-0.96428571, -0.82075472, -0.96226415, -0.99056604, -1.        ,
        -0.97169811, -0.82075472, -0.79545455, -0.97297297, -0.96491228,
        -0.97169811, -0.81132075, -0.98113208],
       [-0.94642857, -0.981818

In [156]:
type(sim_mats[20]) == np.ndarray

True

In [153]:
type(sim_mats[20])

numpy.ndarray

In [126]:
N_labels[N_labels==n].shape

(29,)

In [118]:
copy = sim_mats.copy()

In [140]:
sim_mats = np.zeros(N,dtype='object')
for n in np.unique(N_labels):
    idc = np.where(N_labels==n)[0]
    windows_n = windows[idc]
    if len(windows_n) > 1:
        x = np.zeros((len(windows_n),len(windows_n)))
        for i in range(windows_n.shape[0]):
            for j in range(windows_n.shape[0]):
                common_rows = windows_n[i].intersection(windows_n[j])
                num_identical_rows = len(common_rows)
                x[i,j] = len(common_rows)/min(len(windows_n[i]),len(windows_n[j]))
        np.fill_diagonal(x,0)# make sure the diagonals are zero, this is important the more spikes there are...
        sim_mats[n] = x-1 


In [125]:
sim_mats[n].shape

(29, 29)